# LAB: Árboles de decisión y Bagging

## Introducción

En este lab compararemos el rendimiento de un clasificador de árbol de decisión simple con un clasificador de Bagging. Lo haremos en pocos datasets, empezando por los ofrecidos por Scikit Learn.

Como ya hemos aprendido, el data science es un proceso iterativo. A menudo comenzamos con un modelo muy simple y luego tratamos de mejorar su rendimiento o encontrar mejores modelos para comparar con el inicial.

Este es exactamente el proceso que seguiremos en este laboratorio. Comenzaremos con un modelo simple (Árbol de decisión) y luego compararemos su rendimiento con un modelo Ensamble más complejo.

Usaremos dos datasets:

- para clasificación utilizaremos el [dataset de Cáncer de mama](http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic). Trataremos de diagnosticar un cáncer de mama a partir de pocas características del núcleo celular

- para regressión utilizaremos el [dataset de Diabetes](http://web.stanford.edu/~hastie/Papers/LARS/LeastAngle_2002.pdf), donde trataremos de obtener una medida cuantitativa de la progresión de la enfermedad un año después de la línea de base de 10 variables basales.

## Ejercicio

### Requisitos

1. Usando el dataset de Cáncer de mama, inicializar un clasificador de árbol de decisión y utilizar cross_val_score para evaluar su rendimiento
    - Compare el rendimiento con el clasificador de Bagging
    - Usar pipelines y escalado para ver si el rendimiento mejora
    - Explorar el espacio de parámetros con Grid Search
- Usando el dataset de Diabetes, inicializar un Árbol de Regresión y compararlo con un Regresor de Bagging
    - Comparación simple usando cross_val_score
    - Explorar el espacio de parámetros con Grid Search

**Extra:**

- Repetir el análisis anterior para el dataset de IMDB

## Recursos Adicionales

- [Validación cruzada en el ejemplo de Diabetes](http://scikit-learn.org/stable/auto_examples/exercises/plot_cv_diabetes.html)
- [paper del dataset de Diabetes](http://web.stanford.edu/~hastie/Papers/LARS/LeastAngle_2002.pdf)
- [Clasificador Bagging](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html)
- [Grid Search](http://scikit-learn.org/stable/modules/grid_search.html)

## 1. Dataset de Cáncer de mama
Comenzaremos la comparación con el dataset de cáncer de mama.
Pueden leerlo directamente desde scikit-learn usando la función `load_breast_cancer`.

### 1.a Comparación simple
1. Leer los datos y crear X e y
- Inicializar un árbol de clasificación y usar use cross_val_score para evaluar su rendimiento. Setear la validacion cruzada a 5-folds
- Construir un clasificador de Bagging sobre un clasificador de árbol y usar cross_val_score para evaluar su rendimiento. Setear la validacion cruzada a 5-folds
- ¿Qué resultado es mejor? ¿El resultado es significativamente diferente? ¿Cómo se puede interpretar eso?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X = data['data']
y = data['target']

In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, cross_val_score

dtr = DecisionTreeClassifier()
folds = KFold(n_splits=5, shuffle=True, random_state=1)
scores_dtr = cross_val_score(dtr, X, y, cv=folds, n_jobs=-1, scoring='accuracy')
print(scores_dtr.mean(), scores_dtr.std())

0.929715882627 0.022833714513


In [32]:
from sklearn.ensemble import BaggingClassifier
bgc = BaggingClassifier(DecisionTreeClassifier())

In [33]:
scores_bgc = cross_val_score(bgc, X, y, cv=folds, n_jobs=-1, scoring='accuracy')

In [34]:
print(scores_bgc.mean(), scores_bgc.std())

0.942058686539 0.0274631215336


### 1.b Escalado con pipelines
Como se puede ver, las características no están normalizadas. ¿Mejorará el resultado con normalización?

1. Cree 2 pipelines, con un paso de pre-procesamiento de escalado y luego un árbol de decisión y un árbol de bagging.
- ¿Qué resultado es mejor? ¿El resultado es significativamente diferente? ¿Cómo se puede interpretar eso?
- ¿Los resultados son diferentes de los no escalados?

In [26]:
from sklearn.preprocessing import scale

In [28]:
scaled_X = scale(X)

In [29]:
dtr = DecisionTreeClassifier()
folds = KFold(n_splits=5, shuffle=True, random_state=1)
scores_dtr = cross_val_score(dtr, scaled_X, y, cv=folds, n_jobs=-1, scoring='accuracy')
print(scores_dtr.mean(), scores_dtr.std())

0.931501319671 0.0320034284393


In [30]:
scores_bgc = cross_val_score(bgc, scaled_X, y, cv=folds, n_jobs=-1, scoring='accuracy')
print(scores_bgc.mean(), scores_bgc.std())

0.940257723956 0.0202771272992


### 1.c Grid Search

El Grid search es una excelente manera de mejorar el rendimiento de un clasificador. Vamos a explorar el espacio de parámetros de ambos modelos y ver si podemos mejorar su rendimiento.

1. Inicializar un GridSearchCV con una validación cruzada de 5-fold para el árbol de decisión de clasificación
- Buscar algunos valores de los parámetros para mejorar el resultado del clasificador
- Utilizar todo el conjunto de datos X e y para la prueba
- Comparar el best\_score\_ después de entrenarlo. ¿Es mejor que antes?
- ¿Cómo se compara el resultado de la Grid Search del árbol de decisión con la del árbol de Bagging?
- Inicializar un GridSearchCV con validación cruzada de 5-fold para el clasificador de árbol de Bagging
- Repetir la búsqueda
    - Tener en cuenta que hay que cambiar los nombres de los parámetros para el base_estimator
    - Tener en cuenta que también hay parámetros adicionales para cambiar
    - Tener en cuenta que puede terminar con un espacio de búsqueda muy grande y demandará mucho tiempo
    - Utilizar el parámetro n_jobs para acelerar la búsqueda

- ¿Mejora el resultado para el clasificador de Bagging?
- ¿Qué resultado es mejor? ¿El resultado es significativamente diferente? ¿Cómo se puede interpretar esto?


## 2 Diabetes y regresión

Scikit Learn tiene un dataset de pacientes diabéticos obtenido de este estudio:

http://www4.stat.ncsu.edu/~boos/var.select/diabetes.html
http://web.stanford.edu/~hastie/Papers/LARS/LeastAngle_2002.pdf

Se midieron 442 pacientes diabéticos en 10 variables basales: edad, sexo, índice de masa corporal, presión arterial media y seis mediciones del suero sanguíneo.

El objetivo es una medida cuantitativa de la progresión de la enfermedad un año después de la línea de base.

Repita la comparación anterior entre un DecisionTreeRegressor y una versión de Bagging del mismo.

### 2.a Comparación simple
1. Leer los datos y crear X e y
- Inicializar un Árbol de Regresión y utilizar cross_val_score para evaluar su rendimiento. Establecer la validación cruzada en 5-fold. ¿Qué medida de evaluación utilizará?
- Construir un Árbol de Regresión con Bagging y usar cross_val_score para evaluar su rendimiento. Establecer la validación cruzada en 5-fold.
- ¿Qué resultado es mejor? ¿El resultado es significativamente diferente? ¿Cómo se puede interpretar esto?


### 2.b Grid Search

Repetir el Grid search como arriba:

1. Inicializar un GridSearchCV con 5-fold de validación cruzada para el árbol de Regresión
- Buscar unos pocos valores de los parámetros para mejorar el resultado del regresor
- Utilizar todo el set de datos X e y para la prueba
- Comparar el best\_score\_ después de entrenarlo. ¿Es mejor que antes?
- ¿Cómo se compara el resultado del Grid Search del árbol de decisión con la del árbol de Bagging?
- Inicializar un GridSearchCV con validación cruzada de 5-fold para el árbol de Regresión de Bagging
- Repetir la búsqueda

    - Tener en cuenta que hay que cambiar los nombres de los parámetros para el base_estimator
    - Tener en cuenta que también hay parámetros adicionales para probar
    - Tener en cuenta que puede terminar con un espacio de búsqueda muy grande y demandará mucho tiempo
    - Utilizar el parámetro n_jobs para acelerar la búsqueda

- ¿Mejora el resultado para el Bagging de Regresión?
- ¿Qué resultado es mejor? ¿El resultado es significativamente diferente? ¿Cómo se puede interpretar esto?
